# Klasifikasi Informal-Formal
---

In [ ]:
a = []
while(1):
    a.append('1')

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip -q install conllu nltk transformers tensorflow datasets
!sudo apt-get install xz-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xz-utils is already the newest version (5.2.4-1ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
# Model & Datasets
import tensorflow as tf
from gensim.corpora import WikiCorpus
import torch

# import ktrain

# Regex & Path
import os, re
from pathlib import Path

# NLP
import spacy
import nltk
# nltk.download('all')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from conllu import parse
from conllu import parse_incr
from io import open

import pandas as pd
import numpy as np

from datasets import load_dataset


import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Dataset

In [ ]:
# # 1. Authorizing
# from google.colab import auth
# auth.authenticate_user()

# # 2. Credentials
# import gspread
# from google.auth import default
# creds, _ = default()

# # 3. Authorizing Credentials
# gc = gspread.authorize(creds)

# # 4. Importing Google Sheet using google sheet key
# worksheet = gc.open_by_key('{key}')

# rows = worksheet.get_all_values()


# df = pd.DataFrame.from_records(rows)
# df.head()

# Cola Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/My Drive/Torche Intern NLP Progress/datasets/Indonesian cola dataset v2.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,4,5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Kalimat,Label,Formal Sentences,Paraphrased by,Check,Remarks,Unnamed: 6,Unnamed: 7,Unnamed: 8,Sumber
0,@convomfs astagaaaaaaa tetangga gw emak emak p...,Informal,"Astaga, ibu-ibu tetangga saya memakai sesuatu ...",Ifa,ACCEPTED,NaN,NaN,NaN,NaN,Twitter
1,minimal checkout 120K kepotong 15K wkwkwkwkwkw...,Informal,Minimal lapor keluar Rp 120.000 terpotong menj...,Ifa,ACCEPTED,"""Checkout"" bukan istilah asli dalam Bahasa Ind...",NaN,NaN,NaN,Twitter
2,@hongcages @kdrama_menfess OH iyaa dia yang di...,Informal,"Oh iya, dia pernah jadi pemeran di drama Korea...",Ifa,ACCEPTED WITH COMMENT(S),Jangan dimasukkan ke training data (konteks te...,NaN,NaN,NaN,Twitter
3,Ana mosh adra asada2 eni law etgawezt (b3d el ...,Informal,cuitan berbahasa Arab yang intinya seperti ini...,Ifa,ACCEPTED WITH COMMENT(S),Jangan dimasukkan ke training data (bukan dari...,NaN,NaN,NaN,Twitter
4,rezeki anak yatimðŸ˜‰ https://t.co/rZEmRmE9Z2,Formal,rezeki anak yatim,Shalsa,NaN,NaN,NaN,NaN,NaN,Twitter


In [ ]:
df.describe()

,Unnamed: 6,Unnamed: 7,Unnamed: 8
count,0.0,0.0,0.0
mean,NaN,NaN,NaN
std,NaN,NaN,NaN
min,NaN,NaN,NaN
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182695 entries, 0 to 182694
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Kalimat           179018 non-null  object 
 1   Label             1493 non-null    object 
 2   Formal Sentences  1235 non-null    object 
 3   Paraphrased by    1230 non-null    object 
 4   Check             373 non-null     object 
 5   Remarks           57 non-null      object 
 6   Unnamed: 6        0 non-null       float64
 7   Unnamed: 7        0 non-null       float64
 8   Unnamed: 8        0 non-null       float64
 9   Sumber            2547 non-null    object 
dtypes: float64(3), object(7)
memory usage: 13.9+ MB


In [ ]:
df.drop(
    columns=[
        'Paraphrased by',
        'Unnamed: 6',
        'Unnamed: 7',
        'Unnamed: 8',
        'Sumber'
    ],
    inplace=True
)

In [ ]:
df.shape

(182695, 5)

In [ ]:
df.Label.nunique()

3

In [ ]:
df.groupby(['Label'])['Kalimat'].count()

Label
Formal      544
Formal        1
Informal    948
Name: Kalimat, dtype: int64

# Social Media (WA)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df2 = pd.read_csv('/content/drive/MyDrive/Torche Intern NLP Progress/datasets/WAChat.csv')
df2.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Index,Text,Label
0,0,Halo Kakk ini Nafabillah Safira yang barusan d...,Informal
1,1,ini pid nya ya kakk yang diminta buat jd learn...,Informal
2,2,1 Deskripsi Proses H2 Plant,Informal
3,3,2 Reaksi yang terjadi di masing - masing reactor,Informal
4,4,3 Parameter yang harus diperhatikan pada masin...,Informal


In [ ]:
df2.Label.nunique()

1

In [ ]:
df2.shape

(2971, 3)

In [ ]:
df2.dropna(inplace=True)

In [ ]:
df2.shape

(2971, 3)

# Informal-Formal

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

FormalData = []
InformalData = []

with open("/content/drive/MyDrive/Torche Intern NLP Progress/datasets/FormalLabelling/TrainFormal.txt", 'r', encoding="utf-8") as f:
  FormalData += f.readlines()

with open("/content/drive/MyDrive/Torche Intern NLP Progress/datasets/FormalLabelling/TrainInformal.txt", 'r', encoding="utf-8") as f:
  InformalData += f.readlines()

with open("/content/drive/MyDrive/Torche Intern NLP Progress/datasets/FormalLabelling/Formal.txt", 'r', encoding="utf-8") as f:
  FormalData += f.readlines()

with open("/content/drive/MyDrive/Torche Intern NLP Progress/datasets/FormalLabelling/Informal.txt", 'r', encoding="utf-8") as f:
  InformalData += f.readlines()

with open("/content/drive/MyDrive/Torche Intern NLP Progress/datasets/FormalLabelling/TestFormal.txt", 'r', encoding="utf-8") as f:
  FormalData += f.readlines()

with open("/content/drive/MyDrive/Torche Intern NLP Progress/datasets/FormalLabelling/TestInformal.txt", 'r', encoding="utf-8") as f:
  InformalData += f.readlines()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(len(FormalData))
print(len(InformalData))

2499
2499


# Data Preprocessing

In [ ]:
df = df.dropna(subset=['Label','Kalimat'])
df.shape

(1493, 5)

In [ ]:
df.head()

,Kalimat,Label,Formal Sentences,Check,Remarks
0,@convomfs astagaaaaaaa tetangga gw emak emak p...,Informal,"Astaga, ibu-ibu tetangga saya memakai sesuatu ...",ACCEPTED,NaN
1,minimal checkout 120K kepotong 15K wkwkwkwkwkw...,Informal,Minimal lapor keluar Rp 120.000 terpotong menj...,ACCEPTED,"""Checkout"" bukan istilah asli dalam Bahasa Ind..."
2,@hongcages @kdrama_menfess OH iyaa dia yang di...,Informal,"Oh iya, dia pernah jadi pemeran di drama Korea...",ACCEPTED WITH COMMENT(S),Jangan dimasukkan ke training data (konteks te...
3,Ana mosh adra asada2 eni law etgawezt (b3d el ...,Informal,cuitan berbahasa Arab yang intinya seperti ini...,ACCEPTED WITH COMMENT(S),Jangan dimasukkan ke training data (bukan dari...
4,rezeki anak yatimðŸ˜‰ https://t.co/rZEmRmE9Z2,Formal,rezeki anak yatim,NaN,NaN


In [ ]:
df.Label.unique()

array(['Informal', 'Formal', 'Formal '], dtype=object)

In [ ]:
df['Label'] = df['Label'].replace(['Informal','Formal','Formal '], [1,0,0])
df2['Label'] = df['Label'].replace(['Informal'], [1])

<ipython-input-22-ebd878f6009c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = df['Label'].replace(['Informal','Formal','Formal '], [1,0,0])


In [ ]:
df.groupby(['Label'])['Kalimat'].count()

Label
0    545
1    948
Name: Kalimat, dtype: int64

> normalizing text

In [ ]:
# from nltk.corpus import stopwords

# Init stopwords
# list_stopwords = set(stopwords.words('indonesian'))


# def extract_non_stopwords(text):
#   clean_words = []
#   text = text.split()
#   for word in text:
#       if word not in list_stopwords:
#           clean_words.append(word)
#   return " ".join(clean_words)


def normalize_text(text):
  email_pattern = re.compile(r'[\w._%+-]+@[\w\.-]+\.[a-zA-Z]{2,4}')
  phone_pattern = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4,6}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4,6}|\d{3,4}[-\.\s]??\d{4,6})')
  url_pattern = re.compile(r'www|http:|https:+[^\s]+[\w]')

  text = re.sub(email_pattern, "", text)
  text = re.sub(phone_pattern, "", text)
  text = re.sub(url_pattern, "", text) # Menghapus URL selain https

  text = text.lower()                                   # Mengubah teks menjadi lower case
  text = re.sub(r"[-_()\"#/@;:<>{}`+=~|.!?,]", "", text)
  text = re.sub(r'https?://\S+|www\.\S+', "", text)     # Menghapus URL
  text = re.sub(r'[-+]?[0-9]+', "", text)               # Menghapus angka
  text = re.sub(r'[^\w\s,.]', "", text)                 # Menghapus karakter tanda baca
  text = text.strip()

  text = text.split(".")
  text = [i.split() for i in text]
  text = [" ".join(i)+'.' for i in text]
  text = [i.strip() for i in text]
  return " ".join(text)


# key_norm = pd.read_csv('https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv')

# def text_normalize(text):
#   text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])
#   text = str.lower(text)
#   return text

In [ ]:
cleaned_df = df.copy()
# cleaned_df['Kalimat'] = cleaned_df['Kalimat'].apply(text_normalize)
cleaned_df['Kalimat'] = cleaned_df['Kalimat'].apply(normalize_text)
# cleaned_df['Formal Sentences'] = cleaned_df['Formal Sentences'].apply(normalize_text)
cleaned_df.head()

,Kalimat,Label,Formal Sentences,Check,Remarks
0,convomfs astagaaaaaaa tetangga gw emak emak pa...,1,"Astaga, ibu-ibu tetangga saya memakai sesuatu ...",ACCEPTED,NaN
1,minimal checkout k kepotong k wkwkwkwkwkwkwkwk...,1,Minimal lapor keluar Rp 120.000 terpotong menj...,ACCEPTED,"""Checkout"" bukan istilah asli dalam Bahasa Ind..."
2,hongcages kdramamenfess oh iyaa dia yang di dp...,1,"Oh iya, dia pernah jadi pemeran di drama Korea...",ACCEPTED WITH COMMENT(S),Jangan dimasukkan ke training data (konteks te...
3,ana mosh adra asada eni law etgawezt bd el sha...,1,cuitan berbahasa Arab yang intinya seperti ini...,ACCEPTED WITH COMMENT(S),Jangan dimasukkan ke training data (bukan dari...
4,rezeki anak yatimðÿ.,0,rezeki anak yatim,NaN,NaN


In [ ]:
cleaned_df2 = df2.copy()
# cleaned_df2['Kalimat'] = cleaned_df2['Kalimat'].apply(text_normalize)
cleaned_df2['Text'] = cleaned_df2['Text'].apply(normalize_text)
cleaned_df2.head()

,Index,Text,Label
0,0,halo kakk ini nafabillah safira yang barusan d...,1.0
1,1,ini pid nya ya kakk yang diminta buat jd learn...,1.0
2,2,deskripsi proses h plant.,1.0
3,3,reaksi yang terjadi di masing masing reactor.,1.0
4,4,parameter yang harus diperhatikan pada masing ...,0.0


In [ ]:
FormalData = [normalize_text(i) for i in FormalData if normalize_text(i)]
InformalData = [normalize_text(i) for i in InformalData if normalize_text(i)]


FormalDataLabel = [0] * len(FormalData)
InformalDataLabel = [1] * len(InformalData)

In [ ]:
informal_sentences = list(cleaned_df2['Text'].dropna())
formal_sentences = list(cleaned_df['Formal Sentences'].dropna())
sentences = np.array(list(cleaned_df['Kalimat']) + formal_sentences + informal_sentences + FormalData + InformalData)
label = np.array(list(cleaned_df['Label']) + [0 for i in range(len(formal_sentences))] + [1 for i in range(len(informal_sentences))] + FormalDataLabel + InformalDataLabel)

print(sentences.shape)
print(label.shape)

(10695,)
(10695,)


In [ ]:
np.unique(label, return_counts=True)

(array([0, 1]), array([4277, 6418]))

In [ ]:
!pip install -q imblearn

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(sentences, label, test_size=0.2, random_state=32, shuffle=True)


print(f'Train: X: {X_train.shape} y: {y_train.shape}')
print(f'Validation: X: {X_val.shape} y: {y_val.shape}')



"""
  LSTM
"""
tokenizer_lstm = tf.keras.preprocessing.text.Tokenizer(num_words=512, lower=True)
tokenizer_lstm.fit_on_texts(sentences)

length_pad = tokenizer_lstm.texts_to_sequences(sentences)
length_pad = tf.keras.preprocessing.sequence.pad_sequences(length_pad)

X_train_lstm = tokenizer_lstm.texts_to_sequences(X_train)
X_train_lstm = tf.keras.preprocessing.sequence.pad_sequences(X_train_lstm, maxlen=length_pad.shape[1])

X_val_lstm = tokenizer_lstm.texts_to_sequences(X_val)
X_val_lstm = tf.keras.preprocessing.sequence.pad_sequences(X_val_lstm, maxlen=length_pad.shape[1])

y_train_lstm = tf.one_hot(y_train.copy(), 2)
y_val_lstm = tf.one_hot(y_val.copy(), 2)


print(f'Train LSTM: X: {X_train_lstm.shape} y: {y_train_lstm.shape}')
print(f'Validation LSTM: X: {X_val_lstm.shape} y: {y_val_lstm.shape}')

Train: X: (8556,) y: (8556,)
Validation: X: (2139,) y: (2139,)
Train LSTM: X: (8556, 53) y: (8556, 2)
Validation LSTM: X: (2139, 53) y: (2139, 2)


In [ ]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([3409, 5147]))

In [ ]:
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler

# # sm = SMOTE(sampling_strategy='auto',random_state=32)
# sm = RandomUnderSampler(sampling_strategy='majority', random_state=32)
# X_train, y_train = sm.fit_resample(np.expand_dims(X_train, axis=1), y_train)

# np.unique(y_train, return_counts=True)

# Modelling

In [ ]:
checkpoint_dir = '/content/checkpoints'
if not os.path.exists(checkpoint_dir):
  os.mkdir(checkpoint_dir)

In [ ]:
def visualize_performance(history):
    # Loss
    training_loss = history.history['loss']
    validation_loss = history.history['val_loss']
    epoch = range(1, len(training_loss) + 1)

    # Accuracy
    training_acc = history.history['accuracy']
    validation_acc = history.history['val_accuracy']
    epoch_acc = range(1, len(training_acc) + 1)

    # Precision
    training_pre = history.history['precision']
    validation_pre = history.history['val_precision']
    epoch_pre = range(1, len(training_pre) + 1)

    # Recall
    training_rec = history.history['recall']
    validation_rec = history.history['val_recall']
    epoch_rec = range(1, len(training_rec) + 1)



    fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharey=False)
    # binary crossentropy loss
    axs[0, 0].plot(epoch,training_loss, 'r--')
    axs[0, 0].plot(epoch,validation_loss,'b-')
    axs[0, 0].legend(['Training Loss','Validation Loss'])

    # accuracy
    axs[0, 1].plot(epoch_acc,training_acc, 'r--')
    axs[0, 1].plot(epoch_acc,validation_acc,'b-')
    axs[0, 1].legend(['Training Accuracy','Validation Accuracy'])

    # recall
    axs[1, 0].plot(epoch_rec,training_rec, 'r--')
    axs[1, 0].plot(epoch_rec,validation_rec,'b-')
    axs[1, 0].legend(['Training Recall','Validation Recall'])

    # precision
    axs[1, 1].plot(epoch_pre,training_pre, 'r--')
    axs[1, 1].plot(epoch_pre,validation_pre,'b-')
    axs[1, 1].legend(['Training Precision','Validation Precision'])


    plt.show()

    fig.suptitle('Performa Model')

In [ ]:
from transformers import BertTokenizer, RobertaTokenizer, BertForMaskedLM, RobertaForMaskedLM, RobertaForCausalLM
import tensorflow as tf

# model_name = "indobenchmark/indobert-large-p2"
model_name = "indobenchmark/indobert-base-p2"
model_type = "bert"


batch_size = 64
validation_batch = 8
learning_rate = 3e-4
max_seq_length = 128


regularizer = tf.keras.regularizers.L2(1e-3)


# Class weight balanced

unique_value = np.unique(y_train, return_counts=True)
total = y_train.shape[0]

weight_for_0 = (1 / unique_value[1][0]) * (total / 2.0)
weight_for_1 = (1 / unique_value[1][1]) * (total / 2.0)

class_weight = {unique_value[0][0]: weight_for_0, unique_value[0][1]: weight_for_1}





# Init Opt
optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=learning_rate
)

metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Recall(class_id=1, name='recall'),
    tf.keras.metrics.Precision(class_id=1, name='precision')
]

# loss=tf.keras.losses.BinaryCrossentropy(from_logits=False)
loss = 'categorical_crossentropy'


callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=1
    ),
    # tf.keras.callbacks.ModelCheckpoint(
    #     checkpoint_dir,
    #     monitor='val_loss',
    #     verbose=1,
    #     save_best_only = True,
    #     save_weights_only = True
    # )
]

## BERT

In [ ]:
from transformers import AutoModel, AutoTokenizer, TFBertForSequenceClassification

"""
  BERT
"""
model = TFBertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

input_ids = tf.keras.layers.Input(
    shape=(max_seq_length,),
    dtype=tf.int32,
    name="input_ids"
)
token_type_ids = tf.keras.layers.Input(
    shape=(max_seq_length,),
    dtype=tf.int32,
    name="token_type_ids"
)
attention_mask = tf.keras.layers.Input(
    shape=(max_seq_length,),
    dtype=tf.int32,
    name="attention_mask"
)
x = model.bert(
    input_ids,
    token_type_ids=token_type_ids,
    attention_mask=attention_mask
)[0]

x = tf.keras.layers.Dense(
    128,
    kernel_initializer='he_uniform',
    kernel_regularizer=regularizer
)(x[:,0,:])
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.35)(x)
x = tf.keras.layers.Dense(
    64,
    kernel_initializer='he_uniform',
    kernel_regularizer=regularizer
)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.35)(x)
x = tf.keras.layers.Dense(
    32,
    kernel_initializer='he_uniform',
    kernel_regularizer=regularizer
)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.35)(x)
x = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=x)

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  124441344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]',     

### Train

In [ ]:
def format_return(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, tf.one_hot(label, 2)


def encode_data(text, label=None):
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []

  label = [i for i in label]

  for i in text:
    inputs = tokenizer.encode_plus(
        i,
        add_special_tokens = True,
        padding = 'max_length',
        truncation = 'longest_first',
        max_length = max_seq_length,
        return_attention_mask = True,
        return_token_type_ids=True
    )

    input_ids_list.append(inputs['input_ids'])
    token_type_ids_list.append(inputs['token_type_ids'])
    attention_mask_list.append(inputs['attention_mask'])

  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label)).map(format_return)

In [ ]:
data_train = encode_data(X_train.reshape(-1,), y_train).shuffle(1000).batch(batch_size)
data_val = encode_data(X_val, y_val).batch(validation_batch)

In [ ]:
data_train

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [ ]:
%%time

"""
  Bert
"""

history = model.fit(
    data_train,
    epochs=50,
    verbose=2,
    callbacks=callbacks,
    validation_data=data_val,
    use_multiprocessing=True,
    # class_weight=class_weight
)

visualize_performance(history)

In [ ]:
model.evaluate(data_val)

 39/268 [===>..........................] - ETA: 16:12 - loss: 1.0581 - accuracy: 0.6763 - recall: 0.9951 - precision: 0.6689

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/drive/MyDrive/PA_NLP/Torche Intern NLP Progress/checkpoints/ClassBalanced/BERTCategorical.h5')

## LSTM Bi-directional

In [ ]:
max_embed = 30522

In [ ]:
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph

In [ ]:
regularizer = tf.keras.regularizers.L1L2(1e-4, 1e-3)

In [ ]:
"""
  LSTM
"""

model = tf.keras.Sequential()
model.add(
    tf.keras.layers.Embedding(max_embed, max_seq_length, input_length=X_train_lstm.shape[1])
)
model.add(
    tf.keras.layers.Dropout(0.2)
)
model.add(
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(1024,return_sequences=True)
    )
)
model.add(
    tf.keras.layers.Conv1D(
        512,
        3,
        activation='relu',
        kernel_initializer='he_uniform',
        kernel_regularizer=regularizer
    )
)
model.add(
    tf.keras.layers.Conv1D(
        128,
        1,
        activation='relu',
        kernel_initializer='he_uniform',
        kernel_regularizer=regularizer
    )
)
model.add(
    tf.keras.layers.GlobalMaxPool1D()
)
model.add(
    tf.keras.layers.Dense(
        128,
        activation='relu',
        kernel_initializer='he_uniform',
        kernel_regularizer=regularizer,
    )
)
model.add(
    tf.keras.layers.Dense(
        64,
        activation='relu',
        kernel_initializer='he_uniform',
        kernel_regularizer=regularizer,
    )
)
model.add(
    tf.keras.layers.BatchNormalization()
)
model.add(
    tf.keras.layers.Dropout(0.3)
)
model.add(
    tf.keras.layers.Dense(
        64,
        activation='relu',
        kernel_initializer='he_uniform',
        kernel_regularizer=regularizer,
    )
)
model.add(
    tf.keras.layers.Dense(
        32,
        activation='relu',
        kernel_initializer='he_uniform',
        kernel_regularizer=regularizer,
    )
)
model.add(
    tf.keras.layers.Dense(2, activation='softmax')
)

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

In [ ]:
model.summary()

In [ ]:
%%time


model.load_weights('/content/drive/MyDrive/PA_NLP/Torche Intern NLP Progress/checkpoints/LSTM_weights')


"""
  Bidirectional LSTM
"""
history = model.fit(
    X_train_lstm,
    y_train_lstm,
    epochs=50,
    verbose=2,
    batch_size=batch_size,
    callbacks=callbacks,
    validation_data=(X_val_lstm, y_val_lstm),
    validation_batch_size=validation_batch,
    use_multiprocessing=True,
    class_weight=class_weight
)

In [ ]:
visualize_performance(history)

In [ ]:
model.evaluate(X_val_lstm, y_val_lstm)

In [ ]:
model.save('/content/drive/MyDrive/PA_NLP/Torche Intern NLP Progress/checkpoints/ClassBalanced/BiConvLSTM')

In [ ]:
X_test = tokenizer_lstm.texts_to_sequences(['Yo enggak bisa toh. Kan sudah dibilang sebelumnya, caranya gabisa kalo gitu'])
X_test =  tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=length_pad.shape[1])
result = model.predict(X_test)
print(f'{"Tidak formal" if np.argmax(result) else "formal"}')

In [ ]:
X_test = tokenizer_lstm.texts_to_sequences(['Apakah kamu dapat mengambil barang tersebut?'])
X_test =  tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=length_pad.shape[1])
result = model.predict(X_test)
print(f'{"Tidak formal" if np.argmax(result) else "formal"}')

> Berdasarkan percobaan diatas bila data yang dimasukan tidak memiliki vocab yang sama, maka klasifikasi akan menjadi salah

In [ ]:
y_pred = [np.argmax(i) for i in model.predict(X_val_lstm)]
y_true = [np.argmax(i) for i in y_val_lstm]

print(classification_report(y_true, y_pred))

# LSTM Bi-directional Attention

In [ ]:
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

In [ ]:
from keras import backend as K
"""
  LSTM
"""
max_embed = 30522

input = tf.keras.layers.Input(shape=(X_train_lstm.shape[1],))
embed = tf.keras.layers.Embedding(max_embed, max_seq_length, input_length=X_train_lstm.shape[1])(input)

# x = tf.keras.layers.Dropout(0.2)(embed)
x = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(1024,return_sequences=True)
)(embed)

# Attention
num_heads = 12
atts = []
for _ in range(num_heads):
    att = tf.keras.layers.Attention()([x, x])
    atts.append(att)


x = tf.keras.layers.Concatenate()(atts)
x = tf.keras.layers.Flatten()(x)


x = tf.keras.layers.Dense(2, activation='softmax')(x)



model = tf.keras.Model(
    inputs=input,
    outputs=x
)

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

In [ ]:
model.summary()

In [ ]:
%%time

history = model.fit(
    X_train_lstm,
    y_train_lstm,
    epochs=50,
    verbose=2,
    batch_size=batch_size,
    callbacks=callbacks,
    validation_data=(X_val_lstm, y_val_lstm),
    validation_batch_size=validation_batch,
    use_multiprocessing=True,
    class_weight=class_weight
)

In [ ]:
visualize_performance(history)

In [ ]:
model.evaluate(X_val_lstm, y_val_lstm)

In [ ]:
model.save('/content/drive/MyDrive/PA_NLP/Torche Intern NLP Progress/checkpoints/ClassBalanced/BiConvLSTMSelfAttention')

In [ ]:
X_test = tokenizer_lstm.texts_to_sequences(['Yo enggak bisa toh. Kan sudah dibilang sebelumnya, caranya gabisa kalo gitu'])
X_test =  tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=length_pad.shape[1])
result = model.predict(X_test)
print(f'{"Tidak formal" if np.argmax(result) else "formal"}')

In [ ]:
X_test = tokenizer_lstm.texts_to_sequences(['Apakah kamu dapat mengambil barang tersebut?'])
X_test =  tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=length_pad.shape[1])
result = model.predict(X_test)
print(f'{"Tidak formal" if np.argmax(result) else "formal"}')

In [ ]:
y_pred = [np.argmax(i) for i in model.predict(X_val_lstm)]
y_true = [np.argmax(i) for i in y_val_lstm]

print(classification_report(y_true, y_pred))

> Multi-head Attention:

              precision    recall  f1-score   support

           0       0.76      0.92      0.83       868
           1       0.94      0.80      0.86      1271

    accuracy                           0.85      2139
    macro avg       0.85      0.86      0.85      2139
    weighted avg       0.86      0.85      0.85      2139

In [ ]:
model.evaluate(X_val_lstm, y_val_lstm)

Note: Perlu penambahan data class imbalanced